# Consensus Pharmacophore Tutorial

This tutorial demonstrates how to generate a consensus pharmacophore model from multiple aligned 3D molecular structures. The consensus pharmacophore identifies common pharmacophoric features across different molecules or conformations.

## What is a Consensus Pharmacophore?

A consensus pharmacophore represents the common spatial arrangement of pharmacophoric features shared among multiple molecules. This is useful for:
- Structure-based drug design with multiple ligand poses
- Identifying key interaction points across a series of active compounds
- Virtual screening based on common features

In [ ]:
# Import required libraries
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign, Draw
from pharmacophore import Pharmacophore, View
import warnings
warnings.filterwarnings('ignore')

## Example 1: Consensus from Similar Molecules

We'll use neurotransmitter molecules (Serotonin, Dopamine, Norepinephrine) as examples.

In [ ]:
# Define molecules
smiles_list = [
    "NCCc1c[nH]c2ccc(O)cc12",      # Serotonin
    "NCCc1ccc(O)c(O)c1",            # Dopamine
    "NCC(O)c1ccc(O)c(O)c1"          # Norepinephrine
]

mol_names = ["Serotonin", "Dopamine", "Norepinephrine"]

# Create molecules and add hydrogens
mols = [Chem.MolFromSmiles(s) for s in smiles_list]
mols = [Chem.AddHs(m) for m in mols]

# Generate 3D conformations
ps = AllChem.ETKDGv3()
ps.randomSeed = 42
for m in mols:
    AllChem.EmbedMolecule(m, ps)

## Align Molecules

For consensus pharmacophore generation, molecules should be aligned in 3D space. We'll use Serotonin as the reference.

In [ ]:
# Align molecules to the first one (Serotonin)
aligned_rmsd = []
for i in range(1, len(mols)):
    alignment = rdMolAlign.GetO3A(mols[i], mols[0])
    rmsd = alignment.Align()
    aligned_rmsd.append(rmsd)
    print(f"{mol_names[i]} aligned to {mol_names[0]}, RMSD: {rmsd:.3f} Å")

## Calculate Individual Pharmacophores

First, let's see the pharmacophore features for each molecule individually.

In [ ]:
# Calculate pharmacophores for each molecule
pharm = Pharmacophore()
individual_pharmas = []

for i, mol in enumerate(mols):
    features = pharm.calc_pharm(mol)
    individual_pharmas.append(features)
    print(f"\n{mol_names[i]}: {len(features)} features")
    for f in features:
        print(f"  {f[0]}")

## Generate Consensus Pharmacophore

Now we'll generate the consensus pharmacophore using the `consensus_pharm` method. The distance threshold determines how close features need to be to be merged (default: 2.0 Å).

In [ ]:
# Generate consensus pharmacophore
consensus = pharm.consensus_pharm(mols, distance_threshold=2.0)

print(f"Consensus Pharmacophore: {len(consensus)} features")
print("\nConsensus Features:")

# Group by type
from collections import defaultdict
by_type = defaultdict(list)
for f in consensus:
    by_type[f[0]].append(f)

for feat_type, features in sorted(by_type.items()):
    print(f"\n{feat_type}: {len(features)} feature(s)")
    for f in features:
        print(f"  Position: ({f[2]:.3f}, {f[3]:.3f}, {f[4]:.3f})")

## Visualize with 3D Viewer

We can visualize the aligned molecules with their consensus pharmacophore.

In [ ]:
# Remove hydrogens for clarity in visualization
mols_noH = [Chem.RemoveHs(m) for m in mols]

# Visualize consensus pharmacophore
v = View()
v.view(mols_noH, consensus, labels=True, window=(600, 500))

## Example 2: Effect of Distance Threshold

The distance threshold parameter controls how features are clustered. Let's explore different values.

In [ ]:
# Test different thresholds
thresholds = [1.0, 2.0, 3.0, 4.0]

for threshold in thresholds:
    consensus_temp = pharm.consensus_pharm(mols, distance_threshold=threshold)
    print(f"\nThreshold {threshold:.1f} Å: {len(consensus_temp)} consensus features")
    
    # Count by type
    type_counts = {}
    for f in consensus_temp:
        feat_type = f[0]
        type_counts[feat_type] = type_counts.get(feat_type, 0) + 1
    
    for feat_type, count in sorted(type_counts.items()):
        print(f"  {feat_type}: {count}")

## Example 3: Using RDKit Features

The consensus pharmacophore can also use RDKit's default feature definitions.

In [ ]:
# Generate consensus with RDKit features
consensus_rdkit = pharm.consensus_pharm(mols, distance_threshold=2.0, features='rdkit')

print(f"Consensus Pharmacophore (RDKit): {len(consensus_rdkit)} features")
print("\nFeature types:")
rdkit_types = set(f[0] for f in consensus_rdkit)
for feat_type in sorted(rdkit_types):
    count = sum(1 for f in consensus_rdkit if f[0] == feat_type)
    print(f"  {feat_type}: {count}")

## Export Consensus Pharmacophore

The consensus pharmacophore can be saved for visualization in PyMOL.

In [ ]:
# Output to PyMOL script
pharm.output_features(feature_list=consensus, 
                     savepath='data/consensus_pharma.pml',
                     sphere_size=0.8,
                     transparency=0.3)

## Summary

The `consensus_pharm` method provides a powerful way to identify common pharmacophoric features across multiple molecules:

- **Input**: List of aligned 3D molecules
- **Output**: Consensus pharmacophore features (type and centroid position)
- **Parameters**:
  - `distance_threshold`: Distance in Å for clustering features (default: 2.0)
  - `features`: Feature type - 'default', 'rdkit', or custom dictionary

This is particularly useful for:
1. Structure-based drug design with multiple binding poses
2. Ligand-based pharmacophore modeling
3. Identifying key interaction points for virtual screening